In [1]:
# ====================================================
# directory settings
# ====================================================

import os

OUTPUT_DIR = './'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

TRAIN_PATH = '../input/pgpdataset2022mar07/dataset/train'
TEST_PATH = '../input/pgpdataset2022mar07/dataset/test'

In [2]:
# ====================================================
# CFG
# ====================================================

class CFG:
    print_freq=100
    num_workers = 4
    model_name = 'resnext50_32x4d'
    size = 512
    epochs = 15
    factor = 0.2
    patience = 5
    eps = 1e-6
    lr = 1e-4
    min_lr = 1e-6
    batch_size = 16
    weight_decay = 1e-6
    gradient_accumulation_steps = 1
    max_grad_norm = 1000
    seed = 42
    target_size = 7
    target_col = 'label'
    n_fold = 5
    trn_fold = [1,2,3,4,5]

In [3]:
# ====================================================
# libraries
# ====================================================

import sys
sys.path.append('../input/pytorch-image-models/pytorch-image-models-master')
import os
import math
import time
import random
import shutil
from pathlib import Path
from contextlib import contextmanager
from collections import defaultdict, Counter
import scipy as sp
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.metrics import accuracy_score, f1_score
from tqdm.auto import tqdm
from functools import partial
import cv2
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam, SGD
import torchvision.models as models
from torch.nn.parameter import Parameter
from torch.utils.data import DataLoader, Dataset
from torch.optim.lr_scheduler import ReduceLROnPlateau
from albumentations import (Compose, Normalize, Resize, RandomResizedCrop, HorizontalFlip, VerticalFlip, ShiftScaleRotate, Transpose)
from albumentations.pytorch import ToTensorV2
from albumentations import ImageOnlyTransform
import timm
import warnings 
warnings.filterwarnings('ignore')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
from matplotlib import pyplot as plt
import joblib

In [4]:
# ====================================================
# utils
# ====================================================

def get_score(y_true, y_pred):
    return accuracy_score(y_true, y_pred)

def get_f1_score(y_true, y_pred):
    return f1_score(y_true, y_pred, average='weighted')

@contextmanager
def timer(name):
    t0 = time.time()
    LOGGER.info(f'[{name}] start')
    yield
    LOGGER.info(f'[{name}] done in {time.time() - t0:.0f} s.')

def init_logger(log_file=OUTPUT_DIR+'train.log'):
    from logging import getLogger, INFO, FileHandler,  Formatter,  StreamHandler
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=log_file)
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

LOGGER = init_logger()

def seed_torch(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_torch(seed=CFG.seed)

In [5]:
# ====================================================
# dataset
# ====================================================

class TrainDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        #self.file_names = df['image_ID'].values
        self.labels = df['label'].values
        self.transform = transform
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        file_name = self.df.loc[idx, 'image_ID']
        file_path = f'{TRAIN_PATH}/{file_name}'
        image = Image.open(file_path)
        image = np.array(image)
        if image.shape[-1]>3: image = image[:, :, :-1]
        if image.shape[0]==1:
            image = np.ones((512, 512, 3))
        #print(image.shape)
        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']
        label = torch.tensor(self.labels[idx]).long()
        return image, label

In [6]:
# ====================================================
# transformations
# ====================================================

def get_transforms(*, data):
    
    if data == 'train':
        return Compose([
            RandomResizedCrop(CFG.size, CFG.size),
            Transpose(p=0.5),
            HorizontalFlip(p=0.5),
            VerticalFlip(p=0.5),
            ShiftScaleRotate(p=0.5),
            Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
            ),
            ToTensorV2(),
        ])
    
    elif data == 'valid':
        return Compose([
           Resize(CFG.size, CFG.size),
           Normalize(
               mean=[0.485, 0.456, 0.406],
               std=[0.229, 0.224, 0.225],
           ),
           ToTensorV2(),
       ])

In [7]:
# ====================================================
# model initialization
# ====================================================

class CustomResNext(nn.Module):
    def __init__(self, model_name='resnext50_32x4d', pretrained=False):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained)
        n_features = self.model.fc.in_features
        self.model.fc = nn.Linear(n_features, CFG.target_size)

    def forward(self, x):
        x = self.model(x)
        return x

In [8]:
# ====================================================
# helper functions
# ====================================================

class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (remain %s)' % (asMinutes(s), asMinutes(rs))

In [9]:
def train_fn(train_loader, model, criterion, optimizer, epoch, scheduler, device):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    scores = AverageMeter()
    model.train()
    start = end = time.time()
    global_step = 0
    for step, (images, labels) in enumerate(train_loader):
        data_time.update(time.time() - end)
        if images.shape[0] > 1:            
            images = images.to(device)
            labels = labels.to(device)
            batch_size = labels.size(0)
            y_preds = model(images)
            loss = criterion(y_preds, labels)
            losses.update(loss.item(), batch_size)
            if CFG.gradient_accumulation_steps > 1:
                loss = loss / CFG.gradient_accumulation_steps
            else:
                loss.backward()
            grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), CFG.max_grad_norm)
            if (step + 1) % CFG.gradient_accumulation_steps == 0:
                optimizer.step()
                optimizer.zero_grad()
                global_step += 1
            batch_time.update(time.time() - end)
            end = time.time()
            if step % CFG.print_freq == 0 or step == (len(train_loader)-1):
                print('Epoch: [{0}][{1}/{2}] '
                      'Data {data_time.val:.3f} ({data_time.avg:.3f}) '
                      'Elapsed {remain:s} '
                      'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                      'Grad: {grad_norm:.4f}  '
                      .format(
                       epoch+1, step, len(train_loader), batch_time=batch_time,
                       data_time=data_time, loss=losses,
                       remain=timeSince(start, float(step+1)/len(train_loader)),
                       grad_norm=grad_norm,
                       ))
    return losses.avg

In [10]:
def valid_fn(valid_loader, model, criterion, device):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    scores = AverageMeter()
    model.eval()
    preds = []
    start = end = time.time()
    for step, (images, labels) in enumerate(valid_loader):
        data_time.update(time.time() - end)
        if images.shape[0]>1:
            images = images.to(device)
            labels = labels.to(device)
            batch_size = labels.size(0)
            with torch.no_grad():
                y_preds = model(images)
            loss = criterion(y_preds, labels)
            losses.update(loss.item(), batch_size)
            preds.append(y_preds.softmax(1).to('cpu').numpy())
            if CFG.gradient_accumulation_steps > 1:
                loss = loss / CFG.gradient_accumulation_steps
            batch_time.update(time.time() - end)
            end = time.time()
            if step % CFG.print_freq == 0 or step == (len(valid_loader)-1):
                print('EVAL: [{0}/{1}] '
                      'Data {data_time.val:.3f} ({data_time.avg:.3f}) '
                      'Elapsed {remain:s} '
                      'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                      .format(
                       step, len(valid_loader), batch_time=batch_time,
                       data_time=data_time, loss=losses,
                       remain=timeSince(start, float(step+1)/len(valid_loader)),
                       ))
    predictions = np.concatenate(preds)
    return losses.avg, predictions

In [11]:
# ====================================================
# train loop
# ====================================================

def train_loop(folds, fold):

    LOGGER.info(f"========== fold: {fold} training ==========")

    trn_idx = folds[folds['fold'] != fold].index
    val_idx = folds[folds['fold'] == fold].index

    train_folds = folds.loc[trn_idx].reset_index(drop=True)
    valid_folds = folds.loc[val_idx].reset_index(drop=True)

    train_dataset = TrainDataset(train_folds, transform=get_transforms(data='train'))
    valid_dataset = TrainDataset(valid_folds, transform=get_transforms(data='valid'))

    train_loader = DataLoader(train_dataset, batch_size=CFG.batch_size, 
                              shuffle=True, num_workers=CFG.num_workers, pin_memory=True, drop_last=True)
    valid_loader = DataLoader(valid_dataset, batch_size=CFG.batch_size, 
                              shuffle=False, num_workers=CFG.num_workers, pin_memory=True, drop_last=False)
    
    model = CustomResNext(CFG.model_name, pretrained=True)
    model.to(device)

    optimizer = Adam(model.parameters(), lr=CFG.lr, weight_decay=CFG.weight_decay, amsgrad=False)
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=CFG.factor, patience=CFG.patience, verbose=True, eps=CFG.eps)
  
    criterion = nn.CrossEntropyLoss()

    best_score = 0.
    best_loss = np.inf
    
    for epoch in range(CFG.epochs):
        start_time = time.time()
        avg_loss = train_fn(train_loader, model, criterion, optimizer, epoch, scheduler, device)
        avg_val_loss, preds = valid_fn(valid_loader, model, criterion, device)
        valid_labels = valid_folds[CFG.target_col].values
        scheduler.step(avg_val_loss)
        score = get_score(valid_labels, preds.argmax(1))
        score_f1 = get_f1_score(valid_labels, preds.argmax(1))
        elapsed = time.time() - start_time
        LOGGER.info(f'Epoch {epoch+1} - avg_train_loss: {avg_loss:.4f}  avg_val_loss: {avg_val_loss:.4f}  time: {elapsed:.0f}s')
        LOGGER.info(f'Epoch {epoch+1} - Accuracy: {score}  F1-Score: {score_f1}')
        if score > best_score:
            best_score = score
            LOGGER.info(f'Epoch {epoch+1} - Save Best Score: {best_score:.4f} Model')
            torch.save({'model': model.state_dict(), 'preds': preds}, OUTPUT_DIR+f'{CFG.model_name}_fold{fold}_best.pth')
    
    check_point = torch.load(OUTPUT_DIR+f'{CFG.model_name}_fold{fold}_best.pth')
    valid_folds[[str(c) for c in range(7)]] = check_point['preds']
    valid_folds['preds'] = check_point['preds'].argmax(1)

    return valid_folds

In [12]:
# ====================================================
# main function
# ====================================================

def main():

    def get_result(result_df):
        preds = result_df['preds'].values
        labels = result_df[CFG.target_col].values
        score = get_score(labels, preds)
        LOGGER.info(f'Score: {score:<.5f}')
    
    oof_df = pd.DataFrame()
    for fold in range(CFG.n_fold):
        #if fold in CFG.trn_fold:
        _oof_df = train_loop(folds, fold)
        oof_df = pd.concat([oof_df, _oof_df])
        LOGGER.info(f"========== fold: {fold} result ==========")
        get_result(_oof_df)
    LOGGER.info(f"========== CV ==========")
    get_result(oof_df)
    oof_df.to_csv(OUTPUT_DIR+'oof_df.csv', index=False)

In [13]:
train = pd.read_csv('../input/pgpfolddataset/train.csv')
labels = {'label': {'Cricket': 0, 'Wrestling': 1, 'Tennis': 2, 'Badminton': 3, 'Soccer': 4, 'Swimming': 5, 'Karate': 6}}
train = train.replace(labels)
folds = train

In [14]:
train['label'].value_counts()

0    1556
1    1471
2    1445
3    1394
4    1188
5     595
6     578
Name: label, dtype: int64

In [15]:
main()

========== fold: 0 training ==========
Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/resnext50_32x4d_ra-d733960d.pth" to /root/.cache/torch/hub/checkpoints/resnext50_32x4d_ra-d733960d.pth


Epoch: [1][0/411] Data 1.451 (1.451) Elapsed 0m 9s (remain 64m 10s) Loss: 2.0074(2.0074) Grad: 9.8342  
Epoch: [1][100/411] Data 0.000 (0.015) Elapsed 1m 26s (remain 4m 25s) Loss: 0.7176(1.2674) Grad: 7.7979  
Epoch: [1][200/411] Data 0.000 (0.008) Elapsed 2m 43s (remain 2m 51s) Loss: 1.2781(1.0593) Grad: 18.9191  
Epoch: [1][300/411] Data 0.000 (0.005) Elapsed 4m 0s (remain 1m 28s) Loss: 1.0748(0.9489) Grad: 13.4000  
Epoch: [1][400/411] Data 0.000 (0.004) Elapsed 5m 17s (remain 0m 7s) Loss: 0.6999(0.8775) Grad: 9.1394  
Epoch: [1][410/411] Data 0.000 (0.004) Elapsed 5m 25s (remain 0m 0s) Loss: 0.9273(0.8738) Grad: 11.9485  
EVAL: [0/103] Data 1.238 (1.238) Elapsed 0m 1s (remain 2m 27s) Loss: 0.0751(0.0751) 
EVAL: [100/103] Data 0.000 (0.075) Elapsed 0m 29s (remain 0m 0s) Loss: 0.5155(0.2771) 


Epoch 1 - avg_train_loss: 0.8738  avg_val_loss: 0.2747  time: 356s
Epoch 1 - Accuracy: 0.9113001215066828  F1-Score: 0.9113984041593446
Epoch 1 - Save Best Score: 0.9113 Model


EVAL: [102/103] Data 0.000 (0.074) Elapsed 0m 29s (remain 0m 0s) Loss: 0.1176(0.2747) 
Epoch: [2][0/411] Data 1.211 (1.211) Elapsed 0m 2s (remain 14m 6s) Loss: 0.3580(0.3580) Grad: 5.0363  
Epoch: [2][100/411] Data 0.000 (0.012) Elapsed 1m 19s (remain 4m 3s) Loss: 1.0349(0.5689) Grad: 11.7815  
Epoch: [2][200/411] Data 0.000 (0.006) Elapsed 2m 36s (remain 2m 43s) Loss: 0.4249(0.5618) Grad: 5.8189  
Epoch: [2][300/411] Data 0.000 (0.004) Elapsed 3m 53s (remain 1m 25s) Loss: 0.4739(0.5328) Grad: 6.8897  
Epoch: [2][400/411] Data 0.000 (0.003) Elapsed 5m 10s (remain 0m 7s) Loss: 0.8790(0.5385) Grad: 13.0071  
Epoch: [2][410/411] Data 0.000 (0.003) Elapsed 5m 18s (remain 0m 0s) Loss: 0.3455(0.5354) Grad: 5.4465  
EVAL: [0/103] Data 1.179 (1.179) Elapsed 0m 1s (remain 2m 21s) Loss: 0.0751(0.0751) 
EVAL: [100/103] Data 0.000 (0.057) Elapsed 0m 27s (remain 0m 0s) Loss: 0.6636(0.2189) 


Epoch 2 - avg_train_loss: 0.5354  avg_val_loss: 0.2168  time: 347s
Epoch 2 - Accuracy: 0.9319562575941677  F1-Score: 0.9324189713362732
Epoch 2 - Save Best Score: 0.9320 Model


...

Epoch 14 - avg_train_loss: 0.2255  avg_val_loss: 0.1173  time: 347s
Epoch 14 - Accuracy: 0.9617021276595744  F1-Score: 0.9617280675557517
Epoch 14 - Save Best Score: 0.9617 Model


EVAL: [102/103] Data 0.000 (0.065) Elapsed 0m 28s (remain 0m 0s) Loss: 0.3200(0.1173) 
Epoch: [15][0/411] Data 2.037 (2.037) Elapsed 0m 2s (remain 19m 33s) Loss: 0.2127(0.2127) Grad: 4.9403  
Epoch: [15][100/411] Data 0.001 (0.020) Elapsed 1m 19s (remain 4m 5s) Loss: 0.2682(0.2464) Grad: 6.0461  
Epoch: [15][200/411] Data 0.001 (0.010) Elapsed 2m 36s (remain 2m 43s) Loss: 0.2061(0.2318) Grad: 3.4569  
Epoch: [15][300/411] Data 0.000 (0.007) Elapsed 3m 53s (remain 1m 25s) Loss: 0.1517(0.2330) Grad: 4.5352  
Epoch: [15][400/411] Data 0.000 (0.005) Elapsed 5m 10s (remain 0m 7s) Loss: 0.1323(0.2361) Grad: 4.4070  
Epoch: [15][410/411] Data 0.000 (0.005) Elapsed 5m 18s (remain 0m 0s) Loss: 0.1074(0.2362) Grad: 4.1803  
EVAL: [0/103] Data 0.999 (0.999) Elapsed 0m 1s (remain 2m 3s) Loss: 0.0211(0.0211) 
EVAL: [100/103] Data 0.000 (0.072) Elapsed 0m 29s (remain 0m 0s) Loss: 0.0355(0.1412) 


Epoch 15 - avg_train_loss: 0.2362  avg_val_loss: 0.1421  time: 348s
Epoch 15 - Accuracy: 0.9610942249240122  F1-Score: 0.9611643789203331


EVAL: [102/103] Data 0.000 (0.071) Elapsed 0m 29s (remain 0m 0s) Loss: 0.4239(0.1421) 


========== fold: 4 result ==========
Score: 0.96170
========== CV ==========
Score: 0.96390
